In [1]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from matplotlib.animation import FuncAnimation, PillowWriter
from tqdm.notebook import tqdm
from pycontrails.datalib.ecmwf import ERA5
import cartopy.crs as ccrs

from pycontrails import Flight, Fleet, MetDataset
from pycontrails.models.dry_advection import DryAdvection

In [2]:
df = pd.read_csv("https://apidocs.contrails.org/_static/fleet_sample.csv", parse_dates=["time"])
df["time"] = df["time"].dt.tz_localize(None)

df


,flight_id,engine_uid,aircraft_type,longitude,latitude,altitude,time
0,b5ad83811c43ba5ad584eda8c4b6d836,01P10IA020,A319,-74.178192,40.684994,12.00,2023-02-10 19:48:40
1,b5ad83811c43ba5ad584eda8c4b6d836,01P10IA020,A319,-74.177826,40.685491,12.00,2023-02-10 19:49:00
2,b5ad83811c43ba5ad584eda8c4b6d836,01P10IA020,A319,-74.177460,40.685989,12.00,2023-02-10 19:49:20
3,b5ad83811c43ba5ad584eda8c4b6d836,01P10IA020,A319,-74.177424,40.686048,12.00,2023-02-10 19:49:40
4,b5ad83811c43ba5ad584eda8c4b6d836,01P10IA020,A319,-74.177387,40.686107,12.00,2023-02-10 19:50:00
...,...,...,...,...,...,...,...
184905,80e7604ec96ed36341dd301dd8b6576e,01P10IA025,A321,-112.018956,33.440893,925.00,2023-02-10 23:47:40
184906,80e7604ec96ed36341dd301dd8b6576e,01P10IA025,A321,-112.014633,33.440611,973.25,2023-02-10 23:48:00
184907,80e7604ec96ed36341dd301dd8b6576e,01P10IA025,A321,-112.010310,33.440328,1021.50,2023-02-10 23:48:20
184908,80e7604ec96ed36341dd301dd8b6576e,01P10IA025,A321,-112.005987,33.440046,1069.75,2023-02-10 23:48:40


In [3]:
fl_list = []
for flight_id, group in df.groupby("flight_id"):
    group["altitude"] = group["altitude"] * 0.3048
    aircraft_type = group.pop("aircraft_type").iloc[0]
    flight_id = group.pop("flight_id").iloc[0]
    fl = Flight(group, aircraft_type=aircraft_type, flight_id=flight_id)
    fl_list.append(fl)

fleet = Fleet.from_seq(fl_list)

In [10]:
fleet.time_end.ceil("1H")
fleet.time_start.floor("1H")

Timestamp('2023-02-10 17:00:00')

In [5]:
dt_integration = pd.Timedelta("30min")
max_age = pd.Timedelta("6h")

era5 = ERA5(
        time=(fleet.time_start.floor("1H"), fleet.time_end.ceil("1H") + max_age),
        variables=[
                "t",
                "q",
                "u",
                "v",
                "w",
                "z",
                "relative_humidity"
        ],
        pressure_levels=[1000, 925, 800, 700, 600, 500, 400, 300, 200, 100] 
)

met = era5.open_metdataset()


In [6]:

#met = MetDataset.from_zarr("gs://contrails-301217-ecmwf-era5-zarr/era5_pl.zarr")
#time_sl = slice(fleet.time_start.floor("1H"), fleet.time_end.ceil("1H") + max_age)
#met.data = met.data.sel(time=time_sl)
model = DryAdvection(met, dt_integration=dt_integration, max_age=max_age)
out = model.eval(fleet)

In [7]:
countries = gpd.read_file(
               gpd.datasets.get_path("naturalearth_lowres"))

/tmp/ipykernel_36870/283860368.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  gpd.datasets.get_path("naturalearth_lowres"))


In [8]:
fig, ax = plt.subplots(figsize=(12, 8))

countries.plot(color="lightgrey", ax=ax)
ax.set_xlim(int(out["longitude"].min() - 2), int(out["longitude"].max() + 2))
ax.set_ylim(int(out["latitude"].min() - 2), int(out["latitude"].max() + 2))

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

ax.set_xlabel("Longitude [deg]", fontsize=15)
ax.set_ylabel("Latitude [deg]", fontsize=15)

scat1 = ax.scatter([], [], s=2, color="red", label="Flight path")
scat2 = ax.scatter([], [], c=[], label="Dry advection", alpha=0.5, cmap="viridis", vmin=0, vmax=1)

fig.colorbar(scat2, orientation='horizontal', label='Dry advection', pad=0.2, shrink=0.8)

ax.legend(handles=[scat1], loc='upper left')

# Create a colorbar
#cbar = plt.colorbar(scat2, label='Dry advection', cax=ax)


# Set up legend
# handles, labels = ax.get_legend_handles_labels()
# cmap_legend = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='viridis', markersize=10, label='Colormap')
# handles.append(cmap_legend)
# ax.legend(handles=handles, loc='upper right')

source_frames = fleet.dataframe.groupby(fleet.dataframe["time"].dt.ceil(dt_integration))
dry_adv_frames = out.dataframe.groupby(out.dataframe["time"].dt.ceil(dt_integration))

times = dry_adv_frames.indices

def animate(time):
    ax.set_title(time, fontsize=15)

    try:
        group = source_frames.get_group(time)
    except KeyError:
        offsets = [[None, None]]
    else:
        offsets = group[["longitude", "latitude"]]
    scat1.set_offsets(offsets)

    group = dry_adv_frames.get_group(time)
    offsets = group[["longitude", "latitude"]]
    width = 1e-6 * group["width"]
    color = (group["level"].clip(170, 500) - 170 ) / 330
    scat2.set_offsets(offsets)
    scat2.set_sizes(width)
    scat2.set_array(color)

    return scat1, scat2


plt.close()
ani = FuncAnimation(fig, animate, frames=tqdm(times))
filename = pathlib.Path("advection2.gif")
ani.save(filename, dpi=100, writer=PillowWriter(fps=10))

  0%|          | 0/24 [00:00<?, ?it/s]

In [9]:
from PIL import Image

num_key_frames = 8

with Image.open('advection2.gif') as im:
    for i in range(num_key_frames):
        im.seek(im.n_frames // num_key_frames * i)
        im.save('{}.png'.format(i))